In [1]:
import requests
from pathlib import Path
import os

import shelve

from bs4 import BeautifulSoup
from bs4 import Comment
import pandas as pd

In [2]:
from diskcache import Cache

cache = Cache("players_matches.shelve")

@cache.memoize()
def get_page(link):
    print(link)
    return requests.get(link).content

In [55]:
list(cache.iterkeys())

[('__main__.get_page',
  'https://fbref.com/en/players/15e9c7e6/matchlogs/2014-2015/summary/Nani-Match-Logs'),
 ('__main__.get_page',
  'https://fbref.com/en/players/265ebd37/matchlogs/2014-2015/summary/Nene-Match-Logs'),
 ('__main__.get_page',
  'https://fbref.com/en/players/4a537185/matchlogs/2017-2018/summary/Jese-Match-Logs'),
 ('__main__.get_page',
  'https://fbref.com/en/players/9f351459/matchlogs/2019-2020/summary/Jota-Match-Logs'),
 ('__main__.get_page',
  'https://fbref.com/en/players/b853e0ad/matchlogs/2018-2019/summary/Fred-Match-Logs'),
 ('__main__.get_page',
  'https://fbref.com/en/players/b853e0ad/matchlogs/2019-2020/summary/Fred-Match-Logs'),
 ('__main__.get_page',
  'https://fbref.com/en/players/b853e0ad/matchlogs/2020-2021/summary/Fred-Match-Logs'),
 ('__main__.get_page',
  'https://fbref.com/en/players/f0fc9d0e/matchlogs/2015-2016/summary/Eder-Match-Logs'),
 ('__main__.get_page',
  'https://fbref.com/en/players/3ca7254a/matchlogs/2015-2016/summary/Pedro-Match-Logs'),


In [3]:
def uniquify(df_columns):
    seen = set()

    for item in df_columns:
        fudge = 1
        newitem = item

        while newitem in seen:
            fudge += 1
            newitem = "{}_{}".format(item, fudge)

        yield newitem
        seen.add(newitem)

In [4]:
path = 'dfs/EPL/players'
season_player_match_links = {f.split('_')[0]:os.path.join(path, f) for f in os.listdir(path) if os.path.isfile(os.path.join(path, f)) and f.endswith('match_links.csv')}
season_player_match_links = dict(sorted(season_player_match_links.items(), key = lambda kv:(kv[1], kv[0]), reverse = True))
season_player_match_links

{'2020-2021': 'dfs/EPL/players/2020-2021_match_links.csv',
 '2019-2020': 'dfs/EPL/players/2019-2020_match_links.csv',
 '2018-2019': 'dfs/EPL/players/2018-2019_match_links.csv',
 '2017-2018': 'dfs/EPL/players/2017-2018_match_links.csv',
 '2016-2017': 'dfs/EPL/players/2016-2017_match_links.csv',
 '2015-2016': 'dfs/EPL/players/2015-2016_match_links.csv',
 '2014-2015': 'dfs/EPL/players/2014-2015_match_links.csv'}

In [15]:
dfs = []
for season in season_player_match_links:
    df = pd.read_csv(season_player_match_links[season])
    df = df[df['Rk'] != 'Rk'].reset_index(drop=True)
    dfs.append(df)

In [16]:
dfs[6]

,League,Season,Rk,Player,Nation,Pos,Squad,Age,Born,Matches
0,EPL,2014-2015,1,Patrick van Aanholt,nl NED,DF,Sunderland,23,1990,https://fbref.com/en/players/5f09991f/matchlog...
1,EPL,2014-2015,2,Rolando Aarons,eng ENG,MF,Newcastle Utd,18,1995,https://fbref.com/en/players/c5942695/matchlog...
2,EPL,2014-2015,3,Mehdi Abeid,dz ALG,MF,Newcastle Utd,21,1992,https://fbref.com/en/players/a4c3f7af/matchlog...
3,EPL,2014-2015,4,Charlie Adam,sco SCO,MF,Stoke City,28,1985,https://fbref.com/en/players/aff418dc/matchlog...
4,EPL,2014-2015,5,Emmanuel Adebayor,tg TOG,FW,Tottenham,30,1984,https://fbref.com/en/players/936d9e3e/matchlog...
...,...,...,...,...,...,...,...,...,...,...
543,EPL,2014-2015,544,Bobby Zamora,eng ENG,FW,QPR,33,1981,https://fbref.com/en/players/7ddb3563/matchlog...
544,EPL,2014-2015,545,Mauro Zárate,ar ARG,"FW,MF",West Ham,27,1987,https://fbref.com/en/players/cc18ff4c/matchlog...
545,EPL,2014-2015,546,Mauro Zárate,ar ARG,"FW,MF",QPR,27,1987,https://fbref.com/en/players/cc18ff4c/matchlog...
546,EPL,2014-2015,547,Kurt Zouma,fr FRA,DF,Chelsea,19,1994,https://fbref.com/en/players/ce4246f5/matchlog...


In [53]:
columns = {}
datasets = {}

from tqdm.auto import tqdm

for j, df in tqdm(enumerate(dfs)):
#     if j in [0, 1, 2]:
#         continue
    for i, row in tqdm(df.iterrows()):
        
        data = []
        
        link = row['Matches']
        rk = row['Rk']
        league = row['League']
        season = row['Season']
    
        page = get_page(link)
        
        soup = BeautifulSoup(page, 'html.parser')
        
        column = ['Season'] + [el.text.strip() for el in soup.find('thead').find_all('tr')[1].find_all('th')]
        column = [col+'_Passes' if col == 'Att' else col+'_Dribbles' if col == 'Att_2' else col+'_Passes' if col == 'Prog' else col+'_Carries' if col == 'Prog_2' else col for col in list(uniquify(column))]
        if league not in columns:
            columns[league] = {}
        columns[league][season] = column

        for el in soup.find('div', id='div_matchlogs_all').find('tbody').find_all('tr'):
            row = []
            row.append(season)
            count_empty = 0
            for el2 in el.find_all('th'):
                row.append(el2.text.strip())
                if el2.text.strip() == '':
                    count_empty += 1
            for el2 in el.find_all('td'):
                row.append(el2.text.strip())
                if el2.text.strip() == 'Matches':
                    row[-1] = 'https://fbref.com' + el2.find('a')['href'].strip()
                if el2.text.strip() == '':
                    count_empty += 1
            if row[-2] == 'On matchday squad, but did not play':
                temp = row[-1]
                row[-1] = None
                for i in range(len(columns[league][season]) - len(row) - 1):
                    row.append(None)
                row.append(temp)

            if count_empty != (len(row) - 1):
                data.append(row)
                
        if league not in datasets:
            datasets[league] = {}
        datasets[league][season] = data
        
        df = pd.DataFrame(datasets[league][season], columns = columns[league][season])

        path = Path(f'dfs/{league}/players_matches/{season}')
        path.mkdir(parents=True, exist_ok=True)

        df.to_csv(os.path.join(path, f'player_rk_{rk}.csv'), index = False)

0it [00:00, ?it/s]

0it [00:00, ?it/s]

https://fbref.com/en/players/6421ec64/matchlogs/2017-2018/summary/Xherdan-Shaqiri-Match-Logs
https://fbref.com/en/players/9c94165b/matchlogs/2017-2018/summary/Luke-Shaw-Match-Logs
https://fbref.com/en/players/285a3d44/matchlogs/2017-2018/summary/Ryan-Shawcross-Match-Logs
https://fbref.com/en/players/3727dd3c/matchlogs/2017-2018/summary/Jonjo-Shelvey-Match-Logs
https://fbref.com/en/players/76dd1480/matchlogs/2017-2018/summary/Gylfi-Sigurdsson-Match-Logs
https://fbref.com/en/players/d6f88a1d/matchlogs/2017-2018/summary/Adrien-Silva-Match-Logs
https://fbref.com/en/players/3eb22ec9/matchlogs/2017-2018/summary/Bernardo-Silva-Match-Logs
https://fbref.com/en/players/e2716bd0/matchlogs/2017-2018/summary/David-Silva-Match-Logs
https://fbref.com/en/players/3201b03d/matchlogs/2017-2018/summary/Danny-Simpson-Match-Logs
https://fbref.com/en/players/76fa5f80/matchlogs/2017-2018/summary/Jack-Simpson-Match-Logs
https://fbref.com/en/players/6d6f09dc/matchlogs/2017-2018/summary/Josh-Sims-Match-Logs
http

0it [00:00, ?it/s]

https://fbref.com/en/players/5f09991f/matchlogs/2016-2017/summary/Patrick-van-Aanholt-Match-Logs
https://fbref.com/en/players/aff418dc/matchlogs/2016-2017/summary/Charlie-Adam-Match-Logs
https://fbref.com/en/players/c7174e81/matchlogs/2016-2017/summary/Albert-Adomah-Match-Logs
https://fbref.com/en/players/f76e6b4e/matchlogs/2016-2017/summary/Adrian-Match-Logs
https://fbref.com/en/players/9a8c6673/matchlogs/2016-2017/summary/Ibrahim-Afellay-Match-Logs
https://fbref.com/en/players/b63f4077/matchlogs/2016-2017/summary/Benik-Afobe-Match-Logs
https://fbref.com/en/players/4d034881/matchlogs/2016-2017/summary/Sergio-Aguero-Match-Logs
https://fbref.com/en/players/5c374509/matchlogs/2016-2017/summary/Dan-Agyei-Match-Logs
https://fbref.com/en/players/246d153b/matchlogs/2016-2017/summary/Ola-Aina-Match-Logs
https://fbref.com/en/players/eaeca114/matchlogs/2016-2017/summary/Nathan-Ake-Match-Logs
https://fbref.com/en/players/b827d5b3/matchlogs/2016-2017/summary/Marc-Albrighton-Match-Logs
https://fbr

...

https://fbref.com/en/players/5b54bf3b/matchlogs/2014-2015/summary/Nacer-Chadli-Match-Logs
https://fbref.com/en/players/422862ea/matchlogs/2014-2015/summary/Nathaniel-Chalobah-Match-Logs
https://fbref.com/en/players/790ca5f3/matchlogs/2014-2015/summary/Marouane-Chamakh-Match-Logs
https://fbref.com/en/players/dc6f5bdd/matchlogs/2014-2015/summary/Calum-Chambers-Match-Logs
https://fbref.com/en/players/a772eba6/matchlogs/2014-2015/summary/James-Chester-Match-Logs
https://fbref.com/en/players/c316607a/matchlogs/2014-2015/summary/Vlad-Chiriches-Match-Logs
https://fbref.com/en/players/1cb49278/matchlogs/2014-2015/summary/Andreas-Christensen-Match-Logs
https://fbref.com/en/players/49b05a83/matchlogs/2014-2015/summary/Lee-Chung-yong-Match-Logs
https://fbref.com/en/players/40f3ff51/matchlogs/2014-2015/summary/Papiss-Cisse-Match-Logs
https://fbref.com/en/players/83a2bb30/matchlogs/2014-2015/summary/Aly-Cissokho-Match-Logs
https://fbref.com/en/players/2b91cf8e/matchlogs/2014-2015/summary/Ciaran-Cla

https://fbref.com/en/players/321506b3/matchlogs/2014-2015/summary/Brendan-Galloway-Match-Logs
https://fbref.com/en/players/f27680d9/matchlogs/2014-2015/summary/Cristian-Gamboa-Match-Logs
https://fbref.com/en/players/f744b4df/matchlogs/2014-2015/summary/Dominic-Gape-Match-Logs
https://fbref.com/en/players/41017ad5/matchlogs/2014-2015/summary/Luke-Garbutt-Match-Logs
https://fbref.com/en/players/1117b9ff/matchlogs/2014-2015/summary/Craig-Gardner-Match-Logs
https://fbref.com/en/players/70eb7e9d/matchlogs/2014-2015/summary/Florin-Gardos-Match-Logs
https://fbref.com/en/players/e58a03c6/matchlogs/2014-2015/summary/Dwight-Gayle-Match-Logs
https://fbref.com/en/players/63d17038/matchlogs/2014-2015/summary/Paulo-Gazzaniga-Match-Logs
https://fbref.com/en/players/7ba6d84e/matchlogs/2014-2015/summary/David-de-Gea-Match-Logs
https://fbref.com/en/players/caeb5442/matchlogs/2014-2015/summary/Steven-Gerrard-Match-Logs
https://fbref.com/en/players/1c50f354/matchlogs/2014-2015/summary/Emanuele-Giaccherini

https://fbref.com/en/players/ee15cb07/matchlogs/2014-2015/summary/Cole-Kpekawa-Match-Logs
https://fbref.com/en/players/603cb947/matchlogs/2014-2015/summary/Andrej-Kramaric-Match-Logs
https://fbref.com/en/players/403d2184/matchlogs/2014-2015/summary/Niko-Kranjcar-Match-Logs
https://fbref.com/en/players/9fa0a0b8/matchlogs/2014-2015/summary/Bojan-Krkic-Match-Logs
https://fbref.com/en/players/f23f9d20/matchlogs/2014-2015/summary/Tim-Krul-Match-Logs
https://fbref.com/en/players/cfc41f7a/matchlogs/2014-2015/summary/Daniel-Lafferty-Match-Logs
https://fbref.com/en/players/99813635/matchlogs/2014-2015/summary/Adam-Lallana-Match-Logs
https://fbref.com/en/players/495de82c/matchlogs/2014-2015/summary/Rickie-Lambert-Match-Logs
https://fbref.com/en/players/abe66106/matchlogs/2014-2015/summary/Erik-Lamela-Match-Logs
https://fbref.com/en/players/268df6b1/matchlogs/2014-2015/summary/Frank-Lampard-Match-Logs
https://fbref.com/en/players/b26efc36/matchlogs/2014-2015/summary/Seb-Larsson-Match-Logs
https:/

https://fbref.com/en/players/f716e4e4/matchlogs/2014-2015/summary/Nelson-Oliveira-Match-Logs
https://fbref.com/en/players/48844a9b/matchlogs/2014-2015/summary/Jonas-Olsson-Match-Logs
https://fbref.com/en/players/b28a0b5a/matchlogs/2014-2015/summary/Nedum-Onuoha-Match-Logs
https://fbref.com/en/players/57fe98dd/matchlogs/2014-2015/summary/Oscar-Match-Logs
https://fbref.com/en/players/a014c247/matchlogs/2014-2015/summary/Leon-Osman-Match-Logs
https://fbref.com/en/players/82b1198a/matchlogs/2014-2015/summary/David-Ospina-Match-Logs
https://fbref.com/en/players/31faf295/matchlogs/2014-2015/summary/Bryan-Oviedo-Match-Logs
https://fbref.com/en/players/38c7feef/matchlogs/2014-2015/summary/Alex-Oxlade-Chamberlain-Match-Logs
https://fbref.com/en/players/16380240/matchlogs/2014-2015/summary/Mesut-Ozil-Match-Logs
https://fbref.com/en/players/d24e154c/matchlogs/2014-2015/summary/Costel-Pantilimon-Match-Logs
https://fbref.com/en/players/bb7337cc/matchlogs/2014-2015/summary/Paulinho-Match-Logs
https:

https://fbref.com/en/players/5eecec3d/matchlogs/2014-2015/summary/John-Stones-Match-Logs
https://fbref.com/en/players/1b7ec703/matchlogs/2014-2015/summary/Daniel-Sturridge-Match-Logs
https://fbref.com/en/players/121612e7/matchlogs/2014-2015/summary/Yun-Suk-young-Match-Logs
https://fbref.com/en/players/1d8c99b0/matchlogs/2014-2015/summary/Ki-Sung-yueng-Match-Logs
https://fbref.com/en/players/01b8a2b2/matchlogs/2014-2015/summary/Wojciech-Szczesny-Match-Logs
https://fbref.com/en/players/47dc80f5/matchlogs/2014-2015/summary/Adel-Taarabt-Match-Logs
https://fbref.com/en/players/2475dcc6/matchlogs/2014-2015/summary/Dusan-Tadic-Match-Logs
https://fbref.com/en/players/e514ab62/matchlogs/2014-2015/summary/Matt-Targett-Match-Logs
https://fbref.com/en/players/3b2c9cac/matchlogs/2014-2015/summary/Matthew-Taylor-Match-Logs
https://fbref.com/en/players/c2f9d19f/matchlogs/2014-2015/summary/Neil-Taylor-Match-Logs
https://fbref.com/en/players/5c6dd7bc/matchlogs/2014-2015/summary/Ryan-Taylor-Match-Logs
h